获取新闻地址，再处理

In [99]:
import re

url = 'http://search.youth.cn/cse/search?q=%E4%B9%A0%E8%BF%91%E5%B9%B3%E8%80%83%E5%AF%9F&p=1&s=15107678543080134641&entry=1'

def find_news_nums_page(tree):
    elements = []
    xpath_path = '//div[@id="results"]/div'
    elements.append(tree.xpath(xpath_path))
#     print(elements)
#     print(len(elements[0]))
    return len(elements[0])
    
page_text = requests.get(url=url).text
tree = etree.HTML(page_text)
find_news_nums_page(tree)


10

In [106]:
import requests
from lxml import etree
from newspaper import Article
import pandas as pd

key_names_list = ['习近平']
key_words_1 = ['考察']
key_words_2 = ['企业']

## 测试
words = key_names_list[0]+'%20'+key_words_1+'|'+key_words_2

# 构建搜索xjp的网页
url_1 = 'http://search.youth.cn/cse/search?q='+words+'&p='
# url_2 = 1  #这个是一个数字
url_3 = '&s=15107678543080134641&nsid=&entry=1'

# 构建url
page_num = 501  # 搜索的页面数目
for url_2 in range(1,page_num):
    print("----This is page",url_2)
    url = url_1+str(url_2)+url_3
    print(url)
    page_text = requests.get(url=url).text
    
    tree = etree.HTML(page_text)
    
    urls = []
    
    
    news_num = find_news_nums_page(tree)
    print("Now nums of news is ",news_num)
    
    for i in range(1,news_num):  # news_num为10，从1~10
#         print(i)
        xpath_path = '//*[@id="results"]/div['+str(i)+']/h3/a/@href'
        urls.append(tree.xpath(xpath_path))
    news_title = []
    news_text = []
    news_time = []
    # 此时已经获取了一页的url
    for item in urls:
        print(item[0])
        url_item = item[0]
        paper =Article(url_item, language='zh',memoize_articles=False) 
        paper.download()
        paper.parse()



        news_title.append(paper.title)
        news_time.append(paper.publish_date)
        news_text.append(paper.text)

        print(paper.title)
#         print(paper.publish_date)
#         print(paper.text)

    paper_data = pd.DataFrame({'title':news_title,'time':news_time,'text':news_text})
    
    if url_2 == 1 :
        paper_data.to_csv('Result_500pages.csv') #相对位置，保存在getwcd()获得的路径下 
    else:
        paper_data.to_csv('Result_500pages.csv', mode='a', header=False)



----This is page 1
http://search.youth.cn/cse/search?q=陈敏尔%20考察&p=1&s=15107678543080134641&nsid=&entry=1
Now nums of news is  10
http://news.youth.cn/jsxw/201705/t20170515_9763824_1.htm
当着省委书记的面 5名县委书记登台检讨扶贫工作
http://news.youth.cn/gn/201506/t20150619_6768173.htm
“红色城市”延安遵义数任一把手均高配(图)
http://news.youth.cn/gn/201703/t20170301_9194897_1.htm
【脱贫攻坚看贵州】搬出去 换个活法
http://news.youth.cn/gn/202011/t20201128_12595446.htm
宏伟蓝图鼓舞人心，学习宣讲凝聚力量（深入学习贯彻党的十九届五中全会精神）――学习贯彻党的十九届五中全会精神中央宣讲团宣讲活动综述
http://news.youth.cn/gn/202011/t20201116_12575890.htm
共青团学习宣传贯彻党的十九届五中全会精神要找准着力点提高实效性
http://news.youth.cn/gn/201506/t20150620_6773681.htm


KeyboardInterrupt: 